In [0]:
# imports

import pandas as pd

In [0]:
# model output

PATH_DIR = '/content/drive/My Drive/'

df_prediction = pd.read_csv(PATH_DIR + 'CoClustering_Pred.csv')

df_prediction

,userId,movieId,predicted_rating,true_rating
0,1,307,3.567333,3.5
1,1,1590,2.230047,2.5
2,1,3424,3.395616,4.5
3,2,170,3.266369,3.5
4,2,1296,3.566921,4.5
...,...,...,...,...
5545780,283228,6874,4.483384,4.0
5545781,283228,7072,4.490889,5.0
5545782,283228,7121,4.475352,4.0
5545783,283228,7132,4.620138,4.5


In [0]:
# all movie ids vs titles

df_movies = \
  pd.read_csv(PATH_DIR + 'movies.csv', usecols=[0,1], names=['movieId', 'title'])

all_movies_dict = \
  pd.Series(df_movies['title'].values,index=df_movies['movieId']).to_dict()

In [0]:
# all users in test dataset 

all_users = df_prediction['userId'].unique().tolist()

In [0]:
# results as per predicted ratings in test dataset 

df_prediction1 = df_prediction[['userId', 'movieId', 'predicted_rating']]
df_prediction1['predicted_movies'] = df_prediction1 \
  .apply(lambda x: (x['movieId'], x['predicted_rating']), axis=1)

df_prediction2 = df_prediction1[['userId', 'predicted_movies']]

df_prediction_formatted = df_prediction2 \
  .groupby('userId')['predicted_movies'].apply(list).reset_index(name='recommendation')

df_prediction_sorted = df_prediction_formatted

df_prediction_sorted['recommendation'] = \
  df_prediction_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
  
# df_prediction_sorted

sorted_reco_by_userid = pd.Series( \
  df_prediction_sorted['recommendation'].values,index=df_prediction_sorted['userId']).to_dict()

In [0]:
def get_top_n_recommendations(user, n=10):
  top_n_reco = sorted_reco_by_userid[user][:n]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_reco]

In [0]:
# results as per true ratings in test dataset

df_true1 = df_prediction[['userId', 'movieId', 'true_rating']]
df_true1['movies'] = df_true1 \
  .apply(lambda x: (x['movieId'], x['true_rating']), axis=1)

df_true2 = df_true1[['userId', 'movies']]

df_true_formatted = df_true2 \
  .groupby('userId')['movies'].apply(list).reset_index(name='recommendation')

df_true_sorted = df_true_formatted

df_true_sorted['recommendation'] = \
  df_true_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

# df_true_sorted

sorted_true_ratings_by_userid = pd.Series( \
  df_true_sorted['recommendation'].values,index=df_true_sorted['userId']).to_dict()

In [0]:
def get_top_n_true_ratings(user, n=10):
  top_n_true_ratings = sorted_true_ratings_by_userid[user][:n]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_true_ratings]

In [0]:
# output for each user in test dataset

top_n_recommendations_by_user = {}

user_out = []
for user in all_users:
  out = []

  # user
  out.append(user)

  # top n recommendations (n = 10 by default)
  top_n_reco_for_user = get_top_n_recommendations(user)
  top_n_recommendations_by_user[user] = top_n_reco_for_user

  top_n_true_ratings_for_user = get_top_n_true_ratings(user)
  true_positives = list(set(top_n_reco_for_user) & set(top_n_true_ratings_for_user))
  false_positives = list(set(top_n_reco_for_user) - set(top_n_true_ratings_for_user))
  false_negatives = list(set(top_n_true_ratings_for_user) - set(top_n_reco_for_user))

  # precision for user
  precision_for_user = len(true_positives) / float(len(true_positives) + len(false_positives))
  out.append(precision_for_user)

  # recall for user
  recall_for_user = len(true_positives) / float(len(true_positives) + len(false_negatives))
  out.append(recall_for_user)
  
  user_out.append(out)

In [0]:
# output for all users in test dataset

df_out = pd.DataFrame(user_out, columns=['userId', 'precision_user', 'recall_user'])
df_out

,userId,precision_user,recall_user
0,1,1.0,1.0
1,2,1.0,1.0
2,3,1.0,1.0
3,4,0.2,0.2
4,5,0.6,0.6
...,...,...,...
273652,283224,0.2,0.2
273653,283225,1.0,1.0
273654,283226,1.0,1.0
273655,283227,1.0,1.0


In [0]:
# top n recommendations for user 1

top_n_recommendations_by_user[1]

['Three Colors: Blue (Trois couleurs: Bleu) (1993)',
 'Do the Right Thing (1989)',
 'Event Horizon (1997)']

In [0]:
# EVALUATION

# mae
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_prediction['true_rating'], df_prediction['predicted_rating'])
print('mae', mae)

# rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(df_prediction['true_rating'], df_prediction['predicted_rating']))
print('rmse', rmse)

# precision
precision = df_out['precision_user'].mean()
print('avg precision', precision)

# recall
recall = df_out['recall_user'].mean()
print('avg recall', recall)

# f measure
f_measure = (2.0 * precision * recall) / (precision + recall)
print('f measure', f_measure)

mae 0.6914874633638625
rmse 0.8905791277377583
avg precision 0.8334330932516577
avg recall 0.8334331744563125
f measure 0.8334331338539832
